In [1]:
import os
os.chdir(os.getcwd())

In [2]:
# ------ LIBRARY -------#
import numpy as np
import os
import pickle
import sys
import pandas as pd
import re
import cv2
# torch
import torch
import torch.cuda.amp as amp
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import *

import torch.nn as nn
import torch.nn.functional as F

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau, MultiStepLR, OneCycleLR
#

import math
import torch
from torch.optim.optimizer import Optimizer, required
import torch_optimizer as optim
from collections import defaultdict
import itertools as it

import tqdm
import random
#import time
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import torch
import transformers

# transformer
from transformers import XLMPreTrainedModel, XLMRobertaModel, XLMRobertaConfig, XLMRobertaTokenizer
from transformers import XLMRobertaForSequenceClassification, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification, XLNetForSequenceClassification,\
XLMRobertaForSequenceClassification, XLMForSequenceClassification, RobertaForSequenceClassification
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

In [3]:
train = pd.read_csv("data/train_data.csv")
test = pd.read_csv("data/test_data.csv")
submission = pd.read_csv("data/sample_submission.csv")

In [4]:
train

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,contradiction
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,contradiction
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,entailment
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,neutral
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,neutral
...,...,...,...,...
24993,24993,"오페라에 비하여 오라토리오에서는 독창보다도 합창이 중시되며, 테스토 또는 이스토리쿠...",오라토리오에서 테스토의 역할이 가장 중요하다.,neutral
24994,24994,지하철역까지 걸어서 5분 정도 걸립니다.,지하철역까지 도보로 5분 정도 걸립니다.,entailment
24995,24995,한편 이날 중앙방역대책본부는 집단 감염이 발생한 음식점 관련 역학조사 결과를 공개했다.,중악방역대책본부는 집단 감염과 관련한 모든 정보를 비공개했다.,contradiction
24996,24996,마미손이 랩을 하자 시청자들은 그의 정체를 파악했다.,시청자들은 마미손의 정체를 안다.,entailment


In [5]:
# class args
class args:
    # ---- factor ---- #
    debug=False
    amp = True
    gpu = '1'
    
    epochs=10
    batch_size=16
    weight_decay=1e-6
    n_fold=5
    fold=5 # [0, 1, 2, 3, 4] # 원래는 3
    
    exp_name = 'experiment_name_folder'
    dir_ = f'./saved_models/'
    pt = 'klue/roberta-large'
    max_len = 100
    
    start_lr = 1e-5#1e-3,5e-5
    min_lr=1e-6
    # ---- Dataset ---- #

    # ---- Else ---- #
    num_workers=8
    seed=2021
    scheduler = None#'get_linear_schedule_with_warmup'


data_dir = './'
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
print(device)

##----------------
def set_seeds(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False # for faster training, but not deterministic

set_seeds(seed=args.seed)    

cuda


In [6]:
print(pd.unique(train["label"]))

label_dict = {"entailment" : 0, "contradiction" : 1, "neutral" : 2}

['contradiction' 'entailment' 'neutral']


In [7]:
for i, text in enumerate(train.label):
    train.label[i] = label_dict[text]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
train

,index,premise,hypothesis,label
0,0,"씨름은 상고시대로부터 전해져 내려오는 남자들의 대표적인 놀이로서, 소년이나 장정들이...",씨름의 여자들의 놀이이다.,1
1,1,"삼성은 자작극을 벌인 2명에게 형사 고소 등의 법적 대응을 검토 중이라고 하였으나,...",자작극을 벌인 이는 3명이다.,1
2,2,이를 위해 예측적 범죄예방 시스템을 구축하고 고도화한다.,예측적 범죄예방 시스템 구축하고 고도화하는 것은 목적이 있기 때문이다.,0
3,3,광주광역시가 재개발 정비사업 원주민들에 대한 종합대책을 마련하는 등 원주민 보호에 ...,원주민들은 종합대책에 만족했다.,2
4,4,"진정 소비자와 직원들에게 사랑 받는 기업으로 오래 지속되고 싶으면, 이런 상황에서는...",이런 상황에서 책임 있는 모습을 보여주는 기업은 아주 드물다.,2
...,...,...,...,...
24993,24993,"오페라에 비하여 오라토리오에서는 독창보다도 합창이 중시되며, 테스토 또는 이스토리쿠...",오라토리오에서 테스토의 역할이 가장 중요하다.,2
24994,24994,지하철역까지 걸어서 5분 정도 걸립니다.,지하철역까지 도보로 5분 정도 걸립니다.,0
24995,24995,한편 이날 중앙방역대책본부는 집단 감염이 발생한 음식점 관련 역학조사 결과를 공개했다.,중악방역대책본부는 집단 감염과 관련한 모든 정보를 비공개했다.,1
24996,24996,마미손이 랩을 하자 시청자들은 그의 정체를 파악했다.,시청자들은 마미손의 정체를 안다.,0


In [9]:
def preprocessing(line):
    # line = re.sub(r"[^가-힣a-zA-Z0-9\u4e00-\u9fa5\s]+", " ", line)
    line = re.sub(r"[^가-힣a-zA-Z0-9\s]+", " ", line)
    line = " ".join(filter(lambda word: not word.isdigit(), line.split()))
    line = re.sub(r"\s{2,}", " ", line).strip()
    # line = line.lower()

    if len(line) >= 2 and line[1] == " ":
        line = line[2:].strip()
    return line

In [10]:
tokenizer = AutoTokenizer.from_pretrained(args.pt, use_fast=True)
tokenizer(train['premise'][0], train['hypothesis'][0], max_length=args.max_len, 
                                                pad_to_max_length=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


{'input_ids': [0, 14441, 2073, 12382, 13169, 2200, 3797, 21505, 9005, 2259, 3997, 2031, 2079, 3661, 31221, 5845, 2200, 2112, 16, 5950, 15351, 17788, 7285, 748, 2088, 22048, 2470, 1132, 21893, 15351, 6481, 27135, 5417, 4084, 1972, 2145, 17524, 2138, 15526, 2259, 575, 28674, 18, 2, 14441, 2079, 3883, 2031, 2079, 5845, 28674, 18, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [11]:
def preprocessing_train(data):
    
    pt = args.pt#'monologg/kobert'
    
    tokenizer = AutoTokenizer.from_pretrained(args.pt, use_fast=True)
    
    MAX_LEN = args.max_len
    train=data[['premise', 'hypothesis', 'label']]

#     lines = []
#     for _, row in tqdm.tqdm(train.iterrows()):
#         line = preprocessing(row.premise)
#         lines.append(line)
#     train['premise_pre'] = lines

#     lines = []
#     for _, row in tqdm.tqdm(train.iterrows()):
#         line = preprocessing(row.hypothesis)
#         lines.append(line)
#     train['hypothesis_pre'] = lines

    input_ids =[]
    attention_masks =[]
    token_type_ids =[]
    train_data_labels = []

    for train_premise, train_hypothesis, train_label in tqdm.tqdm(zip(train['premise'], train['hypothesis'], train['label'])):
        try:
            train_sent = train_premise + " " + tokenizer.sep_token + " " + train_hypothesis
            token= tokenizer(train_sent, max_length=args.max_len, 
                                                pad_to_max_length=True)

            input_ids.append(token['input_ids'])
            attention_masks.append(token['attention_mask'])
            token_type_ids.append(token['token_type_ids'])
            #########################################
            train_data_labels.append(train_label)

        except Exception as e:
            print(e)
            pass

    train_input_ids=np.array(input_ids, dtype=int)
    train_attention_masks=np.array(attention_masks, dtype=int)
    train_token_type_ids=np.array(token_type_ids, dtype=int)
    ###########################################################
    train_inputs=(train_input_ids, train_attention_masks, train_token_type_ids)
    train_labels=np.asarray(train_data_labels, dtype=np.int32)

    # save
    train_data = {}

    train_data['input_ids'] = train_input_ids
    train_data['attention_mask'] = train_attention_masks
    train_data['token_type_ids'] = train_token_type_ids
    train_data['targets'] = np.asarray(train_data_labels, dtype=np.int32)
    
    os.makedirs(f'./data/{pt}/', exist_ok=True)
    with open(f'./data/{pt}/train_data_{MAX_LEN}_sep.pickle', 'wb') as f:
        pickle.dump(train_data, f, pickle.HIGHEST_PROTOCOL)


In [12]:
preprocessing_train(train)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
24998it [00:05, 4386.45it/s]


In [13]:
with open(f'./data/{args.pt}/train_data_{args.max_len}_sep.pickle', 'rb') as f:
    train_data = pickle.load(f)

In [14]:
train_data['token_type_ids'][0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [15]:
def preprocessing_test(data):
    
    pt = args.pt#'monologg/kobert'
    
    tokenizer = AutoTokenizer.from_pretrained(args.pt)
    
    MAX_LEN = args.max_len
    test=data[['premise', 'hypothesis']]

#     lines = []
#     for _, row in tqdm.tqdm(test.iterrows()):
#         line = preprocessing(row.premise)
#         lines.append(line)
#     test['premise_pre'] = lines

#     lines = []
#     for _, row in tqdm.tqdm(test.iterrows()):
#         line = preprocessing(row.hypothesis)
#         lines.append(line)
#     test['hypothesis_pre'] = lines
    
    input_ids =[]
    attention_masks =[]
    token_type_ids =[]

    for test_premise, test_hypothesis in tqdm.tqdm(zip(test['premise'], test['hypothesis'])):
        try:
            test_sent = test_premise + " " + tokenizer.sep_token + " " + test_hypothesis
            token= tokenizer(test_sent, max_length=args.max_len, 
                                                pad_to_max_length=True)

            input_ids.append(token['input_ids'])
            attention_masks.append(token['attention_mask'])
            token_type_ids.append(token['token_type_ids'])
            #########################################

        except Exception as e:
            print(e)
            pass


    test_input_ids=np.array(input_ids, dtype=int)
    test_attention_masks=np.array(attention_masks, dtype=int)
    test_token_type_ids=np.array(token_type_ids, dtype=int)
    ###########################################################
    test_inputs=(test_input_ids, test_attention_masks, test_token_type_ids)


    # save
    test_data = {}

    test_data['input_ids'] = test_input_ids
    test_data['attention_mask'] = test_attention_masks
    test_data['token_type_ids'] = test_token_type_ids
    
    os.makedirs(f'./data/{pt}/', exist_ok=True)
    with open(f'./data/{pt}/test_data_{MAX_LEN}_sep.pickle', 'wb') as f:
        pickle.dump(test_data, f, pickle.HIGHEST_PROTOCOL)
           

In [16]:
preprocessing_test(test)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
1666it [00:00, 4511.63it/s]


In [17]:
# ------------------------
#  dataset
# ------------------------
class NliDataSet(Dataset):
    
    def __init__(self, data, test=False):
        
        self.data = data
        self.test = test
        
    def __len__(self):
        
        return self.data['input_ids'].shape[0]
    
    def __getitem__(self,idx):
        
        ids = torch.tensor(self.data['input_ids'][idx], dtype=torch.long)
        mask = torch.tensor(self.data['attention_mask'][idx], dtype=torch.long)
        token_type_ids = torch.tensor(self.data['token_type_ids'][idx], dtype=torch.long)
         
            
        if self.test:
            return {
                'ids': ids,
                'mask': mask,
                'token_type_ids': token_type_ids
            }
        
        else:
            target = torch.tensor(self.data['targets'][idx],dtype=torch.long)

            return {
                    'ids': ids,
                    'mask': mask,
                    'token_type_ids': token_type_ids,
                    'targets': target
                }

In [18]:
# - util - #
def get_learning_rate(optimizer):
    lr=[]
    for param_group in optimizer.param_groups:
        lr +=[ param_group['lr'] ]

    assert(len(lr)==1) #we support only one param_group
    lr = lr[0]

    return lr

def load_data():
    train=pd.read_csv('data/train_data.csv')
    test=pd.read_csv('data/test_data.csv')
    
    #
    train=train[['premise', 'hypothesis', 'label']]
    test=test[['premise', 'hypothesis']]
    #
    from sklearn.model_selection import StratifiedKFold
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    train['fold'] = -1
    for n_fold, (_,v_idx) in enumerate(skf.split(train, train['label'])):
        train.loc[v_idx, 'fold']  = n_fold
    train['id'] = [x for x in range(len(train))]
    
    return train, test

In [27]:
# ------------------------
#  scheduler
# ------------------------

def do_valid(net, valid_loader):

    val_loss = 0
    target_lst = []
    pred_lst = []
    logit = []
    loss_fn = nn.CrossEntropyLoss()

    net.eval()
    start_timer = timer()
    for t, data in enumerate(tqdm.tqdm(valid_loader)):
        ids  = data['ids'].to(device)
        mask  = data['mask'].to(device)
        tokentype = data['token_type_ids'].to(device)
        target = data['targets'].to(device)

        with torch.no_grad():
            if args.amp:
                with amp.autocast():
                    # output
                    output = net(ids, mask)
                    output = output[0]

                    # loss
                    loss = loss_fn(output, target)

            else:
                output = net(ids, mask)#.squeeze(0)
                loss = loss_fn(output, target)
            
            val_loss += loss
            target_lst.extend(target.detach().cpu().numpy())
            pred_lst.extend(output.argmax(dim=1).tolist())
            logit.extend(output.tolist())
            
        val_mean_loss = val_loss / len(valid_loader)
        validation_score = f1_score(y_true=target_lst, y_pred=pred_lst, average='macro')
        validation_acc = accuracy_score(y_true=target_lst, y_pred=pred_lst)
        

    return val_mean_loss, validation_score, validation_acc, logit

def do_predict(net, valid_loader):
    
    val_loss = 0
    pred_lst = []
    logit=[]
    net.eval()
    for t, data in enumerate(tqdm.tqdm(valid_loader)):
        ids  = data['ids'].to(device)
        mask  = data['mask'].to(device)
        tokentype = data['token_type_ids'].to(device)

        with torch.no_grad():
            if args.amp:
                with amp.autocast():
                    # output
                    output = net(ids, mask)[0]

            else:
                output = net(ids, mask)
             
            pred_lst.extend(output.argmax(dim=1).tolist())
            logit.extend(output.tolist())
            
    return pred_lst,logit

def run_train(folds=3):
    out_dir = args.dir_+ f'/fold{args.fold}/{args.exp_name}/'
    os.makedirs(out_dir, exist_ok=True)
    
    # load dataset
    train, test = load_data()    
    with open(f'./data/{args.pt}/train_data_{args.max_len}_sep.pickle', 'rb') as f:
        train_data = pickle.load(f)
    with open(f'./data/{args.pt}/test_data_{args.max_len}_sep.pickle', 'rb') as f:
        test_data = pickle.load(f)    
    
    # split fold
    for n_fold in range(5):
        print(n_fold)
        if n_fold != folds:
            print(f'{n_fold} fold pass'+'\n')
            continue
            
        if args.debug:
            train = train.sample(1000).copy()
            
        print(n_fold)
        
        trn_idx = train[train['fold']!=n_fold]['id'].values
        val_idx = train[train['fold']==n_fold]['id'].values
    

        train_dict = {'input_ids' : train_data['input_ids'][trn_idx] , 'attention_mask' : train_data['attention_mask'][trn_idx] , 
                      'token_type_ids' : train_data['token_type_ids'][trn_idx], 'targets' : train_data['targets'][trn_idx]}
        val_dict = {'input_ids' : train_data['input_ids'][val_idx] , 'attention_mask' : train_data['attention_mask'][val_idx] , 
                      'token_type_ids' : train_data['token_type_ids'][val_idx], 'targets' : train_data['targets'][val_idx]}

        ## dataset ------------------------------------
        train_dataset = NliDataSet(data = train_dict)
        valid_dataset = NliDataSet(data = val_dict)
        trainloader = DataLoader(dataset=train_dataset, batch_size=args.batch_size,
                                 num_workers=8, shuffle=True, pin_memory=True)
        validloader = DataLoader(dataset=valid_dataset, batch_size=args.batch_size, 
                                 num_workers=8, shuffle=False, pin_memory=True)

        ## net ----------------------------------------
        scaler = amp.GradScaler()
        if 'xlm-roberta' in args.pt:
            net = XLMRobertaForSequenceClassification.from_pretrained(args.pt, num_labels = 3) 
        
        elif 'klue/roberta' in args.pt:
            net = AutoModelForSequenceClassification.from_pretrained(args.pt, num_labels = 3) 
        else:
            net = AutoModelForSequenceClassification.from_pretrained(args.pt, num_labels = 3) 

        net.to(device)
        if len(args.gpu)>1:
            net = nn.DataParallel(net)

        # ------------------------
        # loss
        # ------------------------
        loss_fn = nn.CrossEntropyLoss()

        # ------------------------
        #  Optimizer
        # ------------------------
        optimizer = optim.Lookahead(optim.RAdam(filter(lambda p: p.requires_grad,net.parameters()), lr=args.start_lr), alpha=0.5, k=5)

        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = len(trainloader)*args.epochs)
        
        
        # ----
        start_timer = timer()
        best_score = 0

        for epoch in range(1, args.epochs+1):
            train_loss = 0
            valid_loss = 0

            target_lst = []
            pred_lst = []
            lr = get_learning_rate(optimizer)
            print(f'-------------------')
            print(f'{epoch}epoch start')
            print(f'-------------------'+'\n')
            print(f'learning rate : {lr : .6f}')
            for t, data in enumerate(tqdm.tqdm(trainloader)):

                # one iteration update  -------------
                ids  = data['ids'].to(device)
                mask  = data['mask'].to(device)
                tokentype = data['token_type_ids'].to(device)
                target = data['targets'].to(device)

                # ------------
                net.train()
                optimizer.zero_grad()


                if args.amp:
                    with amp.autocast():
                        # output
                        output = net(ids, mask)
                        output = output[0]

                        # loss
                        loss = loss_fn(output, target)
                        train_loss += loss


                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()

                else:
                    # output
                    output = net(ids, mask)

                    # loss
                    loss = loss_fn(output, target)
                    train_loss += loss

                    # update
                    loss.backward()
                    optimizer.step()


                # for calculate f1 score
                target_lst.extend(target.detach().cpu().numpy())
                pred_lst.extend(output.argmax(dim=1).tolist())


                if scheduler is not None:
                    scheduler.step() 
            train_loss = train_loss / len(trainloader)
            train_score = f1_score(y_true=target_lst, y_pred=pred_lst, average='macro')
            train_acc = accuracy_score(y_true=target_lst, y_pred=pred_lst)

            # validation
            valid_loss, valid_score, valid_acc, _ = do_valid(net, validloader)


            if valid_acc > best_score:
                best_score = valid_acc
                best_epoch = epoch
                best_loss = valid_loss

                torch.save(net.state_dict(), out_dir + f'/{folds}f_{epoch}e_{best_score:.4f}_s.pth')
                print('best model saved'+'\n')


            print(f'train loss : {train_loss:.4f}, train f1 score : {train_score : .4f}, train acc : {train_acc : .4f}'+'\n')
            print(f'valid loss : {valid_loss:.4f}, valid f1 score : {valid_score : .4f}, valid acc : {valid_acc : .4f}'+'\n')


        print(f'best valid loss : {best_loss : .4f}'+'\n')
        print(f'best epoch : {best_epoch }'+'\n')
        print(f'best accuracy : {best_score : .4f}'+'\n')
        
def run_predict(model_path):
    ## dataset ------------------------------------
    # load
    with open(f'./data/{args.pt}/test_data_{args.max_len}_sep.pickle', 'rb') as f:
        test_dict = pickle.load(f)
        
    print('test load')
    test_dataset = NliDataSet(data = test_dict, test=True)
    testloader = DataLoader(dataset=test_dataset, batch_size=args.batch_size, 
                             num_workers=8, shuffle=False, pin_memory=True)
    print('set testloader')
    ## net ----------------------------------------
    scaler = amp.GradScaler()
    if 'xlm-roberta' in args.pt:
        net = XLMRobertaForSequenceClassification.from_pretrained(args.pt, num_labels = 3) 
        
    elif 'klue/roberta' in args.pt:
        net = AutoModelForSequenceClassification.from_pretrained(args.pt, num_labels = 3) 
    else:
        net = AutoModelForSequenceClassification.from_pretrained(args.pt, num_labels = 3) 
        
    net.to(device)
    
    if len(args.gpu)>1:
        net = nn.DataParallel(net)

    f = torch.load(model_path)
    net.load_state_dict(f, strict=True)  # True
    print('load saved models')
    # ------------------------
    # validation
    preds, logit = do_predict(net, testloader) #outputs
           
    print('complete predict')
    
    return preds, np.array(logit)
     

In [20]:
"""5fold 전용"""
if __name__ == '__main__':

    args.exp_name = str(args.pt) + '_' + str(args.max_len)
    
    for i in [0,1,2,3,4]: # 5fold
        run_train(folds=i)

0
0


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifi

-------------------
1epoch start
-------------------

learning rate :  0.000010


100%|██████████| 313/313 [00:15<00:00, 20.59it/s]


best model saved

train loss : 0.6260, train f1 score :  0.7022, train acc :  0.7044

valid loss : 0.3336, valid f1 score :  0.8854, valid acc :  0.8858

-------------------
2epoch start
-------------------

learning rate :  0.000009


...

best model saved

train loss : 0.0158, train f1 score :  0.9953, train acc :  0.9953

valid loss : 0.4618, valid f1 score :  0.9121, valid acc :  0.9132

-------------------
9epoch start
-------------------

learning rate :  0.000002


100%|██████████| 313/313 [00:14<00:00, 21.06it/s]


best model saved

train loss : 0.0105, train f1 score :  0.9973, train acc :  0.9973

valid loss : 0.4628, valid f1 score :  0.9137, valid acc :  0.9146

-------------------
10epoch start
-------------------

learning rate :  0.000001


100%|██████████| 313/313 [00:14<00:00, 20.89it/s]


best model saved

train loss : 0.0097, train f1 score :  0.9973, train acc :  0.9973

valid loss : 0.4640, valid f1 score :  0.9140, valid acc :  0.9148

best valid loss :  0.4640

best epoch : 10

best accuracy :  0.9148



In [23]:
def ensemble():
    final_logit=0
    
    _, logit1 = run_predict("./saved_models/fold5/klue/roberta-large_100/0f_10e_0.9204_s.pth")
    _, logit2 = run_predict("./saved_models/fold5/klue/roberta-large_100/1f_5e_0.9146_s.pth")
    _, logit3 = run_predict("./saved_models/fold5/klue/roberta-large_100/2f_8e_0.9108_s.pth")
    _, logit4 = run_predict("./saved_models/fold5/klue/roberta-large_100/3f_9e_0.9054_s.pth")
    _, logit5 = run_predict("./saved_models/fold5/klue/roberta-large_100/4f_10e_0.9148_s.pth")
    final_logit += (logit1+logit2+logit3+logit4+logit5)/5
    
    
    return final_logit


In [28]:
final_logit = ensemble()

test load
set testloader


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifi

load saved models


100%|██████████| 105/105 [00:05<00:00, 20.51it/s]


complete predict
test load
set testloader


...

complete predict
test load
set testloader


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifi

load saved models


100%|██████████| 105/105 [00:05<00:00, 20.88it/s]

complete predict


In [29]:
sub = pd.read_csv("./data/sample_submission.csv")
out = [list(label_dict.keys())[_] for _ in final_logit.argmax(1)]

sub['label'] = out
print(sub)
# preds
sub.to_csv(f'./submission/final_submission_roberta-large.csv', index=False)


      index          label
0         0  contradiction
1         1        neutral
2         2     entailment
3         3  contradiction
4         4  contradiction
...     ...            ...
1661   1661        neutral
1662   1662        neutral
1663   1663        neutral
1664   1664        neutral
1665   1665        neutral

[1666 rows x 2 columns]
